In [63]:
# Import Libraries and Dependencies
import pandas as pd
from pathlib import Path

### 1. Combine and Clean the Data
#### Import CSVs

In [64]:
# Read the CSV files into DataFrames.
S2020_path = Path("Resources/athletic_sales_2020.csv")
Sales_2020_df = pd.read_csv(S2020_path)

S2021_path = Path("Resources/athletic_sales_2021.csv")
Sales_2021_df = pd.read_csv(S2021_path)

In [65]:
# Display the 2020 sales DataFrame
Sales_2020_df.head()

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
0,Foot Locker,1185732,1/1/20,Northeast,New York,New York,Men's Street Footwear,50,1200,600000,300000.00,In-store
1,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,68,83,5644,2426.92,Online
2,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,75,275,206250,61875.00,Outlet
3,Foot Locker,1185732,1/1/20,Northeast,New York,New York,Men's Street Footwear,34,384,13056,6789.12,Outlet
4,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,53,83,4399,1407.68,Outlet


In [66]:
# Display the 2021 sales DataFrame
Sales_2021_df.head()


,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
0,West Gear,1128299,1/1/21,West,California,San Francisco,Men's Athletic Footwear,65,750,487500,121875.00,Outlet
1,West Gear,1128299,1/1/21,West,California,San Francisco,Men's Athletic Footwear,51,233,11883,3208.41,Outlet
2,Kohl's,1189833,1/1/21,Midwest,Montana,Billings,Men's Apparel,50,275,137500,82500.00,Outlet
3,Kohl's,1189833,1/1/21,Midwest,Montana,Billings,Men's Apparel,47,77,3619,2714.25,Online
4,West Gear,1128299,1/1/21,West,California,San Francisco,Men's Athletic Footwear,64,225,14400,5184.00,Online


#### Check the data types of each DataFrame

In [67]:
# Check the 2020 sales data types.
Sales_2020_df.dtypes

retailer             object
retailer_id           int64
invoice_date         object
region               object
state                object
city                 object
product              object
price_per_unit        int64
units_sold            int64
total_sales           int64
operating_profit    float64
sales_method         object
dtype: object

In [68]:
# Check the 2021 sales data types.
Sales_2021_df.dtypes

retailer             object
retailer_id           int64
invoice_date         object
region               object
state                object
city                 object
product              object
price_per_unit        int64
units_sold            int64
total_sales           int64
operating_profit    float64
sales_method         object
dtype: object

#### Combine the sales data by rows.

In [69]:
# Combine the 2020 and 2021 sales DataFrames on the rows and reset the index.
combined_sales_df = pd.concat([Sales_2020_df, Sales_2021_df]).reset_index(drop=True)

In [70]:
# Check if any values are null.
combined_sales_df.isnull().sum()

retailer            0
retailer_id         0
invoice_date        0
region              0
state               0
city                0
product             0
price_per_unit      0
units_sold          0
total_sales         0
operating_profit    0
sales_method        0
dtype: int64

In [71]:
# Check the data type of each column
combined_sales_df.dtypes

retailer             object
retailer_id           int64
invoice_date         object
region               object
state                object
city                 object
product              object
price_per_unit        int64
units_sold            int64
total_sales           int64
operating_profit    float64
sales_method         object
dtype: object

In [72]:
# Convert the "invoice_date" to a datetime datatype
combined_sales_df["invoice_date"] = pd.to_datetime(combined_sales_df["invoice_date"])

C:\Users\mfh30\AppData\Local\Temp\ipykernel_23968\3465559096.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined_sales_df["invoice_date"] = pd.to_datetime(combined_sales_df["invoice_date"])


In [73]:
# Confirm that the "invoice_date" data type has been changed.
combined_sales_df.dtypes

retailer                    object
retailer_id                  int64
invoice_date        datetime64[ns]
region                      object
state                       object
city                        object
product                     object
price_per_unit               int64
units_sold                   int64
total_sales                  int64
operating_profit           float64
sales_method                object
dtype: object

### 2. Determine which Region Sold the Most Products

#### Using `groupby`

In [74]:
# Show the number products sold for region, state, and city.
products_sold_gb = combined_sales_df.groupby(["region", "state", "city"]).count()["units_sold"]
# Rename the sum to "Total_Products_Sold".
products_sold_gb = products_sold_gb.rename("Total_Products_Sold")

# Show the top 5 results.
products_sold_gb.head()
                            

region   state     city        
Midwest  Illinois  Chicago         144
         Indiana   Indianapolis    144
         Iowa      Des Moines      144
         Kansas    Wichita         144
         Michigan  Detroit         144
Name: Total_Products_Sold, dtype: int64

#### Using `pivot_table`

In [75]:
# Show the number products sold for region, state, and city.
products_sold_pt = combined_sales_df.pivot_table(index=["region", "state", "city"], values="units_sold", aggfunc="count")


# Rename the "units_sold" column to "Total_Products_Sold"
products_sold_pt = products_sold_pt.rename(columns={"units_sold": "Total_Products_Sold"})

# Show the top 5 results.
products_sold_pt.head()


Total_Products_Sold
region  state    city                             
Midwest Illinois Chicago                       144
        Indiana  Indianapolis                  144
        Iowa     Des Moines                    144
        Kansas   Wichita                       144
        Michigan Detroit                       144

### 3. Determine which Region had the Most Sales

#### Using `groupby`

In [76]:
# Show the total sales for the products sold for each region, state, and city.
# Rename the "total_sales" column to "Total Sales"
total_sales_gb = combined_sales_df.groupby(["region", "state", "city"])["total_sales"].sum()
total_sales_gb = total_sales_gb.rename("Total Sales")

# Show the top 5 results.
total_sales_gb.head()

region   state     city        
Midwest  Illinois  Chicago          9797488
         Indiana   Indianapolis     8836198
         Iowa      Des Moines       7424011
         Kansas    Wichita          9972864
         Michigan  Detroit         18625433
Name: Total Sales, dtype: int64

#### Using `pivot_table`

In [77]:
# Show the total sales for the products sold for each region, state, and city.
total_sales_pt = combined_sales_df.pivot_table(index=["region", "state", "city"], values="total_sales", aggfunc="sum")

# Optional: Rename the "total_sales" column to "Total Sales"
total_sales_pt = total_sales_pt.rename(columns={"total_sales": "Total Sales"})

# Show the top 5 results.
total_sales_pt.head()

Total Sales
region  state    city                     
Midwest Illinois Chicago           9797488
        Indiana  Indianapolis      8836198
        Iowa     Des Moines        7424011
        Kansas   Wichita           9972864
        Michigan Detroit          18625433

### 4. Determine which Retailer had the Most Sales

#### Using `groupby`

In [78]:
# Show the total sales for the products sold for each retailer, region, state, and city.
# Rename the "total_sales" column to "Total Sales"
retailer_sales_gb = combined_sales_df.groupby(["retailer", "region", "state", "city"])["total_sales"].sum()
retailer_sales_gb = retailer_sales_gb.rename("Total Sales")


# Show the top 5 results.
retailer_sales_gb.head()


retailer  region     state          city      
Amazon    Midwest    Ohio           Columbus      16835873
          Northeast  Maine          Portland       8611395
                     Massachusetts  Boston         4193590
                     New Hampshire  Manchester    10077142
                     Vermont        Burlington    13380463
Name: Total Sales, dtype: int64

#### Using `pivot_table`

In [79]:
# Show the total sales for the products sold for each retailer, region, state, and city.
retailer_sales_pt = combined_sales_df.pivot_table(index=["retailer", "region", "state", "city"], values="total_sales", aggfunc="sum")

# Optional: Rename the "total_sales" column to "Total Sales"
retailer_sales_pt = retailer_sales_pt.rename(columns={"total_sales": "Total Sales"})

# Show the top 5 results.
retailer_sales_pt.head()


Total Sales
retailer region    state         city                   
Amazon   Midwest   Ohio          Columbus       16835873
         Northeast Maine         Portland        8611395
                   Massachusetts Boston          4193590
                   New Hampshire Manchester     10077142
                   Vermont       Burlington     13380463

### 5. Determine which Retailer Sold the Most Women's Athletic Footwear

In [80]:
# Filter the sales data to get the women's athletic footwear sales data.
womens_athletic_footwear_sales = combined_sales_df[combined_sales_df['product'] == "Women's Athletic Footwear"]
womens_athletic_footwear_sales.head()


,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
21,Foot Locker,1185732,2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,36,281,10116,3742.92,Outlet
22,Foot Locker,1185732,2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,41,247,10127,4658.42,Online
23,Foot Locker,1185732,2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,45,850,382500,133875.00,In-store
26,West Gear,1128299,2020-01-05,Northeast,Pennsylvania,Philadelphia,Women's Athletic Footwear,45,300,135000,47250.00,Outlet
27,West Gear,1128299,2020-01-05,Northeast,Pennsylvania,Philadelphia,Women's Athletic Footwear,34,90,3060,1254.60,Outlet


#### Using `groupby`

In [81]:
# Show the total number of women's athletic footwear sold for each retailer, region, state, and city.\
# Rename the "units_sold" column to "Womens_Footwear_Units_Sold"
womens_athletic_footwear_sales_gb = womens_athletic_footwear_sales.groupby(["retailer", "region", "state", "city"])["units_sold"].sum()
womens_athletic_footwear_sales_gb = womens_athletic_footwear_sales_gb.rename("Womens_Footwear_Units_Sold")

# Show the top 5 results.
womens_athletic_footwear_sales_gb.head()

retailer  region     state          city      
Amazon    Midwest    Ohio           Columbus      5801
          Northeast  Maine          Portland      1841
                     Massachusetts  Boston        1190
                     New Hampshire  Manchester    3109
                     Vermont        Burlington    4327
Name: Womens_Footwear_Units_Sold, dtype: int64

#### Using `pivot_table`

In [82]:
# Show the total number of women's athletic footwear sold for each retailer, region, state, and city.
womens_athletic_footwear_sales_pt = womens_athletic_footwear_sales.pivot_table(index=["retailer", "region", "state", "city"], values="units_sold", aggfunc="sum")

# Rename the "units_sold" column to "Womens_Footwear_Units_Sold"
womens_athletic_footwear_sales_pt = womens_athletic_footwear_sales_pt.rename(columns={"units_sold": "Womens_Footwear_Units_Sold"})

# Show the top 5 results.
womens_athletic_footwear_sales_pt.head()

Womens_Footwear_Units_Sold
retailer region    state         city                                  
Amazon   Midwest   Ohio          Columbus                          5801
         Northeast Maine         Portland                          1841
                   Massachusetts Boston                            1190
                   New Hampshire Manchester                        3109
                   Vermont       Burlington                        4327

### 5. Determine the Day with the Most Women's Athletic Footwear Sales

In [83]:
# Create a pivot table with the 'invoice_date' column is the index, and the "total_sales" as the values.
womens_athletic_footwear_sales_day = womens_athletic_footwear_sales.pivot_table(index="invoice_date", values="total_sales", aggfunc="sum")

# Optional: Rename the "total_sales" column to "Total Sales"
womens_athletic_footwear_sales_day = womens_athletic_footwear_sales_day.rename(columns={"total_sales": "Total Sales"})

# Show the table.
womens_athletic_footwear_sales_day.head()


,Total Sales
invoice_date,
2020-01-04,402743
2020-01-05,141801
2020-01-11,129556
2020-01-17,173013
2020-01-22,388250


In [84]:
# Resample the pivot table into daily bins, and get the total sales for each day.
womens_athletic_footwear_sales_day_resampled = womens_athletic_footwear_sales_day.resample("D").sum()


# Sort the resampled pivot table in ascending order on "Total Sales".
womens_athletic_footwear_sales_day_resampled = womens_athletic_footwear_sales_day_resampled.sort_values("Total Sales", ascending=True)

womens_athletic_footwear_sales_day_resampled.tail()


,Total Sales
invoice_date,
2021-07-23,1021806
2021-08-17,1086294
2021-06-17,1376988
2021-12-16,1473497
2021-07-16,1521825


### 6.  Determine the Week with the Most Women's Athletic Footwear Sales

In [85]:
# Resample the pivot table into weekly bins, and get the total sales for each week.
womens_athletic_footwear_sales_week_resampled = womens_athletic_footwear_sales_day.resample("W").sum()

# Sort the resampled pivot table in ascending order on "Total Sales".
womens_athletic_footwear_sales_week_resampled = womens_athletic_footwear_sales_week_resampled.sort_values("Total Sales", ascending=True)

womens_athletic_footwear_sales_week_resampled.tail()

,Total Sales
invoice_date,
2021-11-14,2531721
2021-07-18,2801449
2021-07-11,2835078
2021-12-12,2922161
2021-12-19,3098970
